# Environment

In [1]:
import tensorflow as tf

from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession

# BigQuery Connector

In [2]:
GCP_PROJECT_ID = "axa-ch-machine-learning-dev"
DATASET_GCP_PROJECT_ID = "axa-ch-machine-learning-dev"
DATASET_ID = "nlp_text_classification"
TABLE_ID = "stackoverflow_posts"

In [3]:
#ops.enable_eager_execution()
client = BigQueryClient()

read_session = client.read_session(
    "projects/" + GCP_PROJECT_ID,
    DATASET_GCP_PROJECT_ID,
    TABLE_ID,
    DATASET_ID,
    ["id", "title", "body", "tags"],
    [dtypes.int64, dtypes.string, dtypes.string, dtypes.string],
    requested_streams=2,
    row_restriction="",
)
dataset = read_session.parallel_read_rows()

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


# Model

In [4]:
BATCH_SIZE = 4
BUFFER_SIZE = 100

In [5]:
dataset = dataset.shuffle(buffer_size=BUFFER_SIZE)
dataset = dataset.batch(batch_size=BATCH_SIZE)

In [6]:
iterator = dataset.make_initializable_iterator()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


RuntimeError: dataset.make_initializable_iterator is not supported when eager execution is enabled.

# How get the Data Back?

In [4]:
iter = dataset.make_one_shot_iterator()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [5]:
el = iter.get_next()

In [6]:
el

OrderedDict([('body',
              <tf.Tensor: id=23, shape=(), dtype=string, numpy=b'<p>Handle the value in a try catch block which handles <code>NumberFormatException</code> and if it catches use sth. like <a href= http://stackoverflow.com/questions/3935304/how-to-perform-unsigned-to-signed-conversion-in-java >this</a></p>'>),
             ('id', <tf.Tensor: id=24, shape=(), dtype=int64, numpy=14909625>),
             ('tags', <tf.Tensor: id=25, shape=(), dtype=string, numpy=b''>),
             ('title', <tf.Tensor: id=26, shape=(), dtype=string, numpy=b''>)])

In [7]:
# Data to Model

In [8]:
iter = dataset.make_one_shot_iterator()
x, y, z1, z2 = iter.get_next()

# Batches

In [9]:
def plot_batch_sizes(ds):
    batch_sizes = [batch.shape[0] for batch in ds]
    plt.bar(range(len(batch_sizes)), batch_sizes)
    plt.xlabel("Batch number")
    plt.ylabel("Batch size")

In [10]:
dataset_batches = dataset.repeat(10).batch(128)

In [11]:
plot_batch_sizes(dataset_batches)

AttributeError: 'collections.OrderedDict' object has no attribute 'shape'

In [ ]:
BATCH_SIZE = 4
BUFFER_SIZE = 100
dataset = dataset.shuffle(buffer_size=BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
iter = dataset.make_one_shot_iterator()
el = iter.get_next()

# Model

In [35]:
model = tf.keras.Sequential(
    [
        preprocessing_layer,
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

NameError: name 'preprocessing_layer' is not defined

# Keras